# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2a7c3e6009-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Politicians in E.U.") 

Consider the following exploratory information need:

> You investigating the careers of politicians in the E.U.

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wdt:P106`    | profession   | predicate |
| `wd:Q82955`   | politician | node      |
| `wd:Q46`      | Europe        | node |
| `wd:Q38`      | Italy          | node |



Also consider

```
?p wdt:P106/wdt:P279* wd:Q82955  . 
```

is the BGP to retrieve all **politicians**

## Workload Goals


1. Identify the BGP to retrieve E.U. countries and their politicians

2. Identify the BGP for obtaining other occupations and properties of politicians

3. How many politicians are recorder for each E.U. country?

4. Are there politicians with double citizenship?

5. Analyze the number of politicians in each country by occupation, for instance
 
   5.1 What are the top-3 occupations for a politician in Italy and France?
   
   5.2 What if you consider only politicians for which we don't have a date of death?
   
   5.3 Which politicians had a spouse that was also a politician? How many in each country?


In [1]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P106/wdt:P279 wd:Q82955  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '15556')]


1

In [6]:
#here i find the relationships of a country using the country italy
queryString = """
SELECT ?country ?relation ?relationName ?node
WHERE { 

 



wd:Q38 ?relation ?node.

?relation <http://schema.org/name> ?relationName.
} 
LIMIT 300
"""

print("Results")
run_query(queryString)

Results
[('relation', 'http://www.wikidata.org/prop/direct/P1005'), ('relationName', 'Portuguese National Library ID'), ('node', '106839')]
[('relation', 'http://www.wikidata.org/prop/direct/P1036'), ('relationName', 'Dewey Decimal Classification'), ('node', '2--45')]
[('relation', 'http://www.wikidata.org/prop/direct/P1081'), ('relationName', 'Human Development Index'), ('node', '0.873')]
[('relation', 'http://www.wikidata.org/prop/direct/P1082'), ('relationName', 'population'), ('node', '60317000')]
[('relation', 'http://www.wikidata.org/prop/direct/P1120'), ('relationName', 'number of deaths'), ('node', '633133')]
[('relation', 'http://www.wikidata.org/prop/direct/P1151'), ('relationName', "topic's main Wikimedia portal"), ('node', 'http://www.wikidata.org/entity/Q10631821')]
[('relation', 'http://www.wikidata.org/prop/direct/P1198'), ('relationName', 'unemployment rate'), ('node', '12')]
[('relation', 'http://www.wikidata.org/prop/direct/P122'), ('relationName', 'basic form of gove

300

In [ ]:
#i found the relationship member of trougth the BGP  P463 and the european unionwich has the BGP Q458

In [7]:
#Identify the BGP to retrieve E.U. countries and their politicians
queryString = """
SELECT DISTINCT ?politician ?country ?cname ?pname
WHERE { 
?country wdt:P463 wd:Q458.
?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 ?country.
?country <http://schema.org/name> ?cname.
?politician <http://schema.org/name> ?pname.
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('politician', 'http://www.wikidata.org/entity/Q1654394'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('pname', 'Patrick Buisson')]
[('politician', 'http://www.wikidata.org/entity/Q26251285'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('pname', 'René Bidal')]
[('politician', 'http://www.wikidata.org/entity/Q48321355'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('pname', 'Stéphane Séjourné')]
[('politician', 'http://www.wikidata.org/entity/Q1369960'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('pname', 'Hugues Aubriot')]
[('politician', 'http://www.wikidata.org/entity/Q354672'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('pname', 'Peter II of Courtenay')]
[('politician', 'http://www.wikidata.org/entity/Q28964087'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('pname', 'Sophia Chikirou')]
[('politic

10

In [8]:
#Identify the BGP for obtaining other occupations and properties of politicians in the E.U. I'm assuming we are considering politicians inside the E.U.
#this is the BGP for obtaining all the profession of the politicians
queryString = """
SELECT DISTINCT ?politician ?profession ?professionname ?pname
WHERE { 
?country wdt:P463 wd:Q458.
?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 ?country.
?politician wdt:P106 ?profession.

?profession <http://schema.org/name> ?professionname.
?politician <http://schema.org/name> ?pname.
}
LIMIT 10

"""

print("Results")
run_query(queryString)


Results
[('politician', 'http://www.wikidata.org/entity/Q19301647'), ('profession', 'http://www.wikidata.org/entity/Q66775776'), ('professionname', 'commercial expert'), ('pname', 'Ramon Coma i Casellas')]
[('politician', 'http://www.wikidata.org/entity/Q99534387'), ('profession', 'http://www.wikidata.org/entity/Q703534'), ('professionname', 'employee'), ('pname', 'Claudine Rey')]
[('politician', 'http://www.wikidata.org/entity/Q1654394'), ('profession', 'http://www.wikidata.org/entity/Q8125919'), ('professionname', 'political adviser'), ('pname', 'Patrick Buisson')]
[('politician', 'http://www.wikidata.org/entity/Q19301647'), ('profession', 'http://www.wikidata.org/entity/Q8125919'), ('professionname', 'political adviser'), ('pname', 'Ramon Coma i Casellas')]
[('politician', 'http://www.wikidata.org/entity/Q21780326'), ('profession', 'http://www.wikidata.org/entity/Q8125919'), ('professionname', 'political adviser'), ('pname', 'Cristina López González')]
[('politician', 'http://www.wi

10

In [9]:
#Identify the BGP for obtaining other occupations and properties of politicians in the E.U.
#this is the BGP for obtaining all the profession of the politicians and all their other propreties
queryString = """
SELECT DISTINCT ?politician ?profession ?professionname ?pname ?propreties
WHERE { 
?country wdt:P463 wd:Q458.
?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 ?country.
?politician wdt:P106 ?profession.
?politician ?propreties ?object.

?profession <http://schema.org/name> ?professionname.
?propreties <http://schema.org/name> ?pname.
}
LIMIT 100

"""

print("Results")
run_query(queryString)

Results
[('politician', 'http://www.wikidata.org/entity/Q19301647'), ('profession', 'http://www.wikidata.org/entity/Q66775776'), ('professionname', 'commercial expert'), ('pname', 'member of political party'), ('propreties', 'http://www.wikidata.org/prop/direct/P102')]
[('politician', 'http://www.wikidata.org/entity/Q19301647'), ('profession', 'http://www.wikidata.org/entity/Q66775776'), ('professionname', 'commercial expert'), ('pname', 'occupation'), ('propreties', 'http://www.wikidata.org/prop/direct/P106')]
[('politician', 'http://www.wikidata.org/entity/Q19301647'), ('profession', 'http://www.wikidata.org/entity/Q66775776'), ('professionname', 'commercial expert'), ('pname', 'languages spoken, written or signed'), ('propreties', 'http://www.wikidata.org/prop/direct/P1412')]
[('politician', 'http://www.wikidata.org/entity/Q19301647'), ('profession', 'http://www.wikidata.org/entity/Q66775776'), ('professionname', 'commercial expert'), ('pname', 'place of birth'), ('propreties', 'htt

100

In [10]:
#How many politicians are recorder for each E.U. country?
queryString = """
SELECT DISTINCT COUNT(?politician) ?cname
WHERE { 
?country wdt:P463 wd:Q458.
?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 ?country.



?country <http://schema.org/name> ?cname.
}GROUP BY(?cname)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1302'), ('cname', 'Slovenia')]
[('callret-0', '1452'), ('cname', 'Estonia')]
[('callret-0', '5324'), ('cname', 'Poland')]
[('callret-0', '2281'), ('cname', 'Czech Republic')]
[('callret-0', '82748'), ('cname', 'France')]
[('callret-0', '4471'), ('cname', 'Hungary')]
[('callret-0', '33607'), ('cname', 'Spain')]
[('callret-0', '8137'), ('cname', 'Belgium')]
[('callret-0', '4859'), ('cname', 'Greece')]
[('callret-0', '12411'), ('cname', 'United Kingdom')]


10

In [11]:

#Are there EU politicians with double citizenship?
#this returnthe politician whith the country look at the followings queries to find politicians whit double citizenship
queryString = """
SELECT DISTINCT ?country  ?politician ?cname
WHERE { 
?country wdt:P463 wd:Q458.
?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 ?country.

?country <http://schema.org/name> ?cname.
}
LIMIT 10

"""

print("Results")
run_query(queryString)












Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('politician', 'http://www.wikidata.org/entity/Q1654394'), ('cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('politician', 'http://www.wikidata.org/entity/Q26251285'), ('cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('politician', 'http://www.wikidata.org/entity/Q48321355'), ('cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('politician', 'http://www.wikidata.org/entity/Q1369960'), ('cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('politician', 'http://www.wikidata.org/entity/Q354672'), ('cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('politician', 'http://www.wikidata.org/entity/Q28964087'), ('cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('politician', 'http://www.wikidata.org/entity/Q51719'), ('cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('politi

10

In [13]:

#this return the politicians who have more than 1 citizenship


queryString = """


SELECT DISTINCT ?politician
WHERE { 
 ?politician wdt:P27 ?country.
 ?country wdt:P463 wd:Q458.
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 ?politician wdt:P27 ?country1.
 ?politician wdt:P27 ?country2.
 

  

  
  
  
 
  FILTER(?country1!=?country2)
}LIMIT 20
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': [], 'vars': ['politician']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q211681'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q63886'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q480717'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q4721414'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q4977632'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q3108729'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q19276856'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q3434633'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q6514674'}},
   {'politician': {'type': 'uri',
     'value': 'http://

In [14]:
#Are there EU politicians with double citizenship?


queryString = """

ASK WHERE{
SELECT DISTINCT ?politician 
WHERE { 
 ?politician wdt:P27 ?country.
 ?country wdt:P463 wd:Q458.
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 ?politician wdt:P27 ?country1.
 ?politician wdt:P27 ?country2.
 

  

  
  
  
 
  FILTER(?country1!=?country2)


} 
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

In [ ]:
# What are the top-3 occupations for a politician in Italy and France?

In [9]:
#the BGP for italy is wd:Q38, the one for france is wd:Q142 already found in the first notebook
#return politician and their occupation
queryString = """
SELECT DISTINCT ?politician ?professionname  
WHERE {
{

?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 wd:Q142.
?politician wdt:P106 ?profession.
?politician ?propreties ?object.

?profession <http://schema.org/name> ?professionname.

}
UNION
{

?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 wd:Q38.
?politician wdt:P106 ?profession.
?politician ?propreties ?object.

?profession <http://schema.org/name> ?professionname.

}
}
LIMIT 30

"""

print("Results")
run_query(queryString)

Results
[('politician', 'http://www.wikidata.org/entity/Q1654394'), ('professionname', 'political adviser')]
[('politician', 'http://www.wikidata.org/entity/Q48321355'), ('professionname', 'political adviser')]
[('politician', 'http://www.wikidata.org/entity/Q28964087'), ('professionname', 'political adviser')]
[('politician', 'http://www.wikidata.org/entity/Q65131729'), ('professionname', 'political adviser')]
[('politician', 'http://www.wikidata.org/entity/Q28964087'), ('professionname', 'communications adviser')]
[('politician', 'http://www.wikidata.org/entity/Q51719'), ('professionname', 'feudatory')]
[('politician', 'http://www.wikidata.org/entity/Q721440'), ('professionname', 'feudatory')]
[('politician', 'http://www.wikidata.org/entity/Q26251285'), ('professionname', 'prefect')]
[('politician', 'http://www.wikidata.org/entity/Q8772'), ('professionname', 'prefect')]
[('politician', 'http://www.wikidata.org/entity/Q1369960'), ('professionname', 'Intendant des finances')]
[('politi

30

In [7]:

#return the top profession for politicians in italy and france including the profession politician
queryString = """
SELECT DISTINCT COUNT(distinct ?politician) AS ?number ?professionname 
WHERE {
{

?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 wd:Q38.
?politician wdt:P106 ?profession.
?politician ?propreties ?object.

?profession <http://schema.org/name> ?professionname.

}
UNION
{

?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 wd:Q142.
?politician wdt:P106 ?profession.
?politician ?propreties ?object.

?profession <http://schema.org/name> ?professionname.

}
}GROUP BY (?professionname)
ORDER BY DESC(?number)
LIMIT 3

"""

print("Results")
run_query(queryString)

Results
[('number', '91897'), ('professionname', 'politician')]
[('number', '14819'), ('professionname', 'pensioner')]
[('number', '5433'), ('professionname', 'anciens cadres')]


3

In [8]:
#What if you consider only politicians for which we don't have a date of death?
#here i get the top profession for politicians who don't have a date of death whose BGP I found in the previous queries and it is P570
queryString = """
SELECT DISTINCT COUNT(distinct ?politician) AS ?number ?professionname 
WHERE {
{

?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 wd:Q38.
?politician wdt:P106 ?profession.
?politician ?propreties ?object.

?profession <http://schema.org/name> ?professionname.
FILTER NOT EXISTS{?politician wdt:P570 ?death}.
}
UNION
{

?politician wdt:P106/wdt:P279* wd:Q82955  . 
?politician wdt:P27 wd:Q142.
?politician wdt:P106 ?profession.
?politician ?propreties ?object.

?profession <http://schema.org/name> ?professionname.
FILTER NOT EXISTS{?politician wdt:P570 ?death}.
}
}GROUP BY (?professionname)
ORDER BY DESC(?number)
LIMIT 3

"""

print("Results")
run_query(queryString)

Results
[('number', '63602'), ('professionname', 'politician')]
[('number', '14708'), ('professionname', 'pensioner')]
[('number', '5433'), ('professionname', 'anciens cadres')]


3

In [ ]:
#Which politicians had a spouse that was also a politician? How many in each country? the BGP for spouse is P26 found in the previous queries

In [15]:
#SBAGLIATA VAI ALLA PROSSIMA
#Which politicians had a spouse that was also a politician? I return the EU politician which have a spouse which is also a politician


queryString = """


SELECT DISTINCT ?politician ?spouse
WHERE { 

  ?country wdt:P463 wd:Q458.
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 ?politician wdt:P27 ?country.
 ?politician wdt:P26 ?spouse.
 ?spouse wdt:P106/wdt:P279* wd:Q82955  .
 
} 
LIMIT 10
"""

print("Results")
run_ask_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42000 Error TN...: Exceeded 1000000000 bytes in transitive temp memory.  use t_distinct, t_max or more T_MAX_memory options to limit the search or increase the pool\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-2a7c3e6009-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\n\n\nSELECT DISTINCT ?politician ?spouse\nWHERE { \n\n  ?country wdt:P463 wd:Q458.\n ?politician wdt:P106/wdt:P279* wd:Q82955  . \n ?politician wdt:P27 ?country.\n ?politician wdt:P26 ?spouse.\n ?spouse wdt:P106/wdt:P279* wd:Q82955  .\n \n} \nLIMIT 10\n'


In [2]:

#penso sia corretta ma ci mette troppo tempo rifare
#VAI ALLA PROSSIMA
#Which politicians had a spouse that was also a politician, How many in each country?


queryString = """


SELECT DISTINCT COUNT(?politician) ?country
WHERE { 

  ?country wdt:P463 wd:Q458.
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 ?politician wdt:P27 ?country.
 ?politician wdt:P26 ?spouse.
 ?spouse wdt:P106/wdt:P279* ?profession  .
 FILTER(?profession=wd:Q82955)
} GROUP BY (?country)
LIMIT 1
"""

print("Results")
run_ask_query(queryString)


Results
The operation failed timed out


In [3]:

#Which politicians had a spouse that was also a politician, How many in each country?
queryString = """


SELECT DISTINCT COUNT(distinct ?politician) ?country
WHERE { 

 ?country wdt:P463 wd:Q458.
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 ?politician wdt:P27 ?country.
 ?politician wdt:P26 ?spouse.
 ?spouse wdt:P106/wdt:P279* wd:Q82955  .
 
} GROUP BY (?country)
LIMIT 10
"""

print("Results")
run_ask_query(queryString)


Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42000 Error TN...: Exceeded 1000000000 bytes in transitive temp memory.  use t_distinct, t_max or more T_MAX_memory options to limit the search or increase the pool\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-2a7c3e6009-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\n\n\nSELECT DISTINCT COUNT(distinct ?politician) ?country\nWHERE { \n\n ?country wdt:P463 wd:Q458.\n ?politician wdt:P106/wdt:P279* wd:Q82955  . \n ?politician wdt:P27 ?country.\n ?politician wdt:P26 ?spouse.\n ?spouse wdt:P106/wdt:P279* wd:Q82955  .\n \n} GROUP BY (?country)\nLIMIT 10\n'


In [4]:
#here i try to get the politician who have a spouse which is also a politician
queryString = """


SELECT DISTINCT ?politician
WHERE { 

  
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 
 ?politician wdt:P26 ?spouse.
 ?spouse wdt:P106/wdt:P279* wd:Q82955  .
 
} 
LIMIT 1
"""

print("Results")
run_ask_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42000 Error TN...: Exceeded 1000000000 bytes in transitive temp memory.  use t_distinct, t_max or more T_MAX_memory options to limit the search or increase the pool\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-2a7c3e6009-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\n\n\nSELECT DISTINCT ?politician\nWHERE { \n\n  \n ?politician wdt:P106/wdt:P279* wd:Q82955  . \n \n ?politician wdt:P26 ?spouse.\n ?spouse wdt:P106/wdt:P279* wd:Q82955  .\n \n} \nLIMIT 1\n'


In [9]:
#here i get the number of italian politician who get a spouse which was also a politician

queryString = """

SELECT COUNT(?politician) AS ?number{
SELECT DISTINCT ?politician
WHERE { 


 
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 ?politician wdt:P27 wd:Q38.
 ?politician wdt:P26 ?spouse.
 ?spouse wdt:P106/wdt:P279* wd:Q82955  .
 
} 
}

"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': [], 'vars': ['number']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'number': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'value': '78'}}]}}

In [11]:
#here i get the French politician who get a spouse which was also a politician

queryString = """


SELECT DISTINCT ?politician
WHERE { 


 
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 ?politician wdt:P27 wd:Q142.
 ?politician wdt:P26 ?spouse.
 ?spouse wdt:P106/wdt:P279* wd:Q82955  .
 
 
}LIMIT 10

"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': [], 'vars': ['politician']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q721440'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q354672'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q3291630'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q99535161'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q99534387'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q16948142'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q577795'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q2105'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q517'}},
   {'politician': {'type': 'uri',
     'value': 'http://www.

In [6]:
#here i get the number of french politicin who have a spouse which is also a politician
queryString = """

SELECT COUNT(?politician) AS ?number{
SELECT DISTINCT ?politician
WHERE { 


 
 ?politician wdt:P106/wdt:P279* wd:Q82955  . 
 ?politician wdt:P27 wd:Q142.
 ?politician wdt:P26 ?spouse.
 
 
 FILTER EXISTS   {?spouse wdt:P106/wdt:P279* wd:Q82955  .}
}
}
"""

print("Results")
run_ask_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42000 Error TN...: Exceeded 1000000000 bytes in transitive temp memory.  use t_distinct, t_max or more T_MAX_memory options to limit the search or increase the pool\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-2a7c3e6009-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\n\nSELECT COUNT(?politician) AS ?number{\nSELECT DISTINCT ?politician\nWHERE { \n\n\n \n ?politician wdt:P106/wdt:P279* wd:Q82955  . \n ?politician wdt:P27 wd:Q142.\n ?politician wdt:P26 ?spouse.\n \n \n FILTER EXISTS   {?spouse wdt:P106/wdt:P279* wd:Q82955  .}\n}\n}\n'
